In [378]:
import re
import math
import numpy as np
import itertools

input_file = open("input.txt")
input = input_file.read().strip().split('\n')
input_file.close()

tile_size = 10
tiles = []

tile_id = None
tile_matrix = []

# PART 1

def get_tile_sides(id, state_id, matrix):
    t = ''.join(matrix[0])
    r = ''.join(np.array(matrix).transpose()[-1])
    b = ''.join(matrix[-1])
    l = ''.join(np.array(matrix).transpose()[0])
    
    return {
        'id': id,
        'state_id': state_id,
        
        't': t,
        'r': r,
        'b': b,
        'l': l,
        
        'all': [t, r, b, l]
    }

class Tile:
    def __init__(self, id, matrix):
        self.id = id
        self.matrix = matrix
        
        self.build_available_states()
    
    def build_available_states(self):
        self.available_states = []
        self.available_positions = []
        
        for i in range(0, 4):
            self.available_states.append(get_tile_sides(self.id, i, np.rot90(self.matrix, i)))
            self.available_positions.append(np.rot90(self.matrix, i))
    
        flipped_matrix = np.fliplr(self.matrix)
        
        for i in range(0, 4):
            self.available_states.append(get_tile_sides(self.id, i + 4, np.rot90(flipped_matrix, i)))
            self.available_positions.append(np.rot90(flipped_matrix, i))
    
    def get_states_by_rules(self, rules):
        matching_states = []
        
        for state in self.available_states:
            is_match = True
            
            for rule in rules:
                if rule['value'] is None:
                    if not rule['side'] in state['sides']:
                        is_match = False
                    continue
                
                if state[rule['side']] != rule['value']:
                    is_match = False
                
            if is_match:
                matching_states.append(state)
            
        return matching_states
    
    def has_side(self, side):
        for state in self.available_states:
            if side in state['all']:
                return True
        
        return False
    
    def __str__(self):
        return str(self.id)

for line in input:
    if 'Tile' in line:
        tile_id = int(re.findall('(\d+)', line)[0])
        continue
    
    if line == '':
        continue
        
    tile_matrix.append(list(line))
    
    if len(tile_matrix) == tile_size:
        tiles.append(Tile(tile_id, tile_matrix))
        tile_matrix = []

def get_initial_tile_states():
    initial_tiles = []
    
    for tile_to_check in tiles:
        for tile_state in tile_to_check.available_states:

            can_be_first = True

            for tile in tiles:
                if tile_to_check.id == tile.id:
                    continue

                if tile.has_side(tile_state['l']) or tile.has_side(tile_state['t']):
                    can_be_first = False
                    break
            
            if can_be_first:
                initial_tiles.append(tile_state)
                    
    return initial_tiles

state_path = []
id_path = []
current_index = 0

exploration_tree = [
    get_initial_tile_states()
]

grid_side_length = int(math.sqrt(len(tiles)))

def get_next_tile_rules(index):
    is_first_column = (index % grid_side_length) == 0
    is_first_row = index < grid_side_length
    
    rules = []
    
    if not is_first_column:
        rules.append({
            'side': 'l',
            'value': state_path[index-1]['r']
        })
    
    if not is_first_row:
        rules.append({
            'side': 't',
            'value': state_path[index-grid_side_length]['b']
        })
    
    return rules

def get_tile_states_by_matching_rules(rules):
    matching_states = []
    
    for tile in tiles:
        if (tile.id in id_path):
            continue
        
        matching_states.extend(tile.get_states_by_rules(rules))
        
    return matching_states

while current_index < len(tiles):
    
    if len(exploration_tree[current_index]) == 0:
        exploration_tree.pop()
        state_path.pop()
        id_path.pop()
        
        current_index -= 1
        
        continue
        
    next_state = exploration_tree[current_index].pop()
    state_path.append(next_state)
    id_path.append(next_state['id'])
    current_index += 1
    
    next_tile_states = get_tile_states_by_matching_rules(get_next_tile_rules(current_index))
        
    exploration_tree.append(next_tile_states)

print(id_path[0] * id_path[grid_side_length-1] * id_path[-1] * id_path[-grid_side_length])

# PART 2

monster_top = '                  # '
monster_middle = '#    ##    ##    ###'
monster_bottom = ' #  #  #  #  #  #   '

monster_re = re.compile('('+ monster_top_re + '.*?\n.*?' + monster_middle_re + '.*?\n.*?' + monster_bottom_re + ')')

tile_map = {}

for tile in tiles:
    tile.matrix = np.asarray(tile.matrix)[1:-1,1:-1].tolist()
    tile.build_available_states()
    tile_map[tile.id] = tile

images = []
    
for state in state_path:
    tile = tile_map[state['id']]
    image = tile.available_positions[state['state_id']]
    images.append(image)

y = 0
x = 0

combined_image = []

while y < grid_side_length:
    row = []
    
    while x < grid_side_length:
        if len(row) == 0:
            row = images[grid_side_length * y + x]
        else:
            row = np.concatenate((row, images[grid_side_length * y + x]), axis=1)
        
        x += 1
    
    if len(combined_image) == 0:
        combined_image = row
    else:
        combined_image = np.concatenate((combined_image, row), axis=0)
    
    x = 0
    y += 1
    
def count_monsters(image):
    image = list(map(lambda x : ''.join(x), image))
    
    count = 0
    
    for i in range(0, len(image)-2):
        section_to_check = [
            image[i],
            image[i+1],
            image[i+2]
        ]
        
        section_to_check = '\n'.join(section_to_check)
    
        if monster_re.search(section_to_check):
            count += 1

    return count

max_count = 0

for i in range(0, 4):
    count = count_monsters(np.rot90(combined_image, i))
    
    if count > max_count:
        max_count = count

flipped_image = np.fliplr(combined_image)

for i in range(0, 4):
    count = count_monsters(np.rot90(flipped_image, i))
    
    if count > max_count:
        max_count = count
        
answer = sum(char == '#' for char in ''.join(map(lambda x : ''.join(x), combined_image))) - 15 * max_count
        
print(answer)

8272903687921
2304
